# Init

In [33]:
import warnings
warnings.filterwarnings('ignore')

!pip install scikit-learn pandas numpy xarray matplotlib
!pip install 'ipywidgets==8.1.7' 'nbformat>=5.10.4' dash ipykernel git+https://github.com/quantiacs/toolbox.git

!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
! make
!make install
!pip install Ta-Lib

# test import
import talib

from IPython.display import Javascript
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'))

  Cloning https://github.com/quantiacs/toolbox.git to /tmp/pip-req-build-8yv47s1c
  Running command git clone --filter=blob:none --quiet https://github.com/quantiacs/toolbox.git /tmp/pip-req-build-8yv47s1c
  Resolved https://github.com/quantiacs/toolbox.git to commit 9f50f71ff1859e079fc99a0a8d5ae5a5b682317c
  Preparing metadata (setup.py) ... done
--2025-11-16 18:55:27--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 104.18.13.149, 104.18.12.149, 2606:4700::6812:c95, ...
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|104.18.13.149|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2025-11-16 18:55:27--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sou

<IPython.core.display.Javascript object>

# Keys

In [34]:
import os

os.environ['API_KEY'] = "ef14002f-13fc-4fd1-9fd2-6e4a6447426f"
os.environ['NONINTERACT'] = 'True'

# Strategy

In [42]:
import logging

import xarray as xr  # xarray for data manipulation

import qnt.data as qndata     # functions for loading data
import qnt.backtester as qnbt # built-in backtester
import qnt.ta as qnta         # technical analysis library
import qnt.stats as qnstats   # statistical functions
import qnt.output as qnout

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

np.seterr(divide = "ignore")

# Note: The original code imported these but didn't use them;
# keeping them for completeness but they are not strictly necessary
# for the provided model logic.
# from qnt.ta.macd import macd
# from qnt.ta.rsi  import rsi
# from qnt.ta.stochastic import stochastic_k, stochastic, slow_stochastic

from sklearn.ensemble import RandomForestRegressor # (Random Forest Regressor)
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error

def get_model():
    # El Regresor de Bosque Aleatorio (Random Forest Regressor) se elige como una mejora sobre los Modelos Lineales
    # (como Bayesian Ridge) debido a que los movimientos de los precios de las acciones son inherentemente no lineales
    # y a menudo dependen de interacciones complejas entre las características (features). Los Bosques Aleatorios,
    # siendo un método de conjunto (ensemble method) basado en árboles de decisión, pueden capturar de forma natural
    # estas relaciones no lineales y son generalmente más robustos ante valores atípicos (outliers)
    # y el sobreajuste (overfitting) que un solo árbol de decisión.
    model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

    return model

# loading s&p-500 stock data for training
assets = [
    "NAS:MSFT",  # Tecnologia/Software Gigante tecnológico estable en software y nube (Azure). Contraste clave con empresas centradas en el consumidor.
    "NAS:NVDA",  # Semiconductores/IA Representa el alto crecimiento en el sector de IA y semiconductores. Es sensible a avances tecnológicos y sentimiento de mercado.
    "NAS:GOOGL", # (Alphabet Inc.) Líder en publicidad digital y búsquedas. Sus movimientos reflejan el gasto en línea y el riesgo regulatorio.
    "NAS:TSLA",  # Automotor Acción de alta volatilidad y basada en el sentimiento. Desafía al modelo con datos "ruidosos" y menos ligados a métricas tradicionales.
    "NYS:JPM",  # Financiero Valor cíclico financiero influenciado por factores macroeconómicos, como las tasas de interés y la política de la Reserva Federal.
    "NYS:WMT"   # Ventas a Consumidor Acción defensiva (bienes de consumo esenciales) con menor volatilidad. Proporciona un punto de referencia para la resistencia económica.
]
data = qndata.stocks.load_spx_data(tail = 365 *5, assets = assets)
[asset for asset in qndata.stocks_load_spx_list() if 'NAS:'+asset['symbol'] in assets]
[asset for asset in qndata.stocks_load_spx_list() if 'NYS:'+asset['symbol'] in assets]

def get_features(data):

    # past closing price:
    price = data.sel(field="close")
    price = price.shift(time=1)

    # High, Low data shifted to avoid look-ahead bias
    high = data.sel(field="high").shift(time=1)
    low = data.sel(field="low").shift(time=1)

    # Existing Features
    trend = qnta.roc(qnta.lwma(price, 60), 1)

    macd2_line, macd2_signal, macd2_hist = qnta.macd(data, 12, 26, 9)

    volatility = qnta.tr(high, low, price)
    volatility = volatility / price
    volatility = qnta.lwma(volatility, 14)

    k, d = qnta.stochastic(high, low, price, 14)
    stochastic_d = d/100

    rsi = qnta.rsi(price)
    rsi = rsi/100

    price_current = data.sel(field="close")
    price_future = price_current.shift(time=-1)
    daily_return = (price_future - price_current)/price_current
    daily_return = daily_return.shift(time=1)

    daily_return1  = daily_return
    daily_return2  = daily_return.shift(time=1)
    daily_return3  = daily_return.shift(time=2)
    daily_return4  = daily_return.shift(time=3)
    daily_return5  = daily_return.shift(time=4)

    # --- Nuevas Características ---

    # 1. Fuerza Relativa (vs. SMA Larga): Mide qué tan fuerte es el precio comparado con su promedio de 50 días.
    sma_long = qnta.sma(price, 50)
    relative_strength_long = price / sma_long

    # 2. Cruce de Medias Móviles (Corta vs. Larga): Mide la relación entre la tendencia de corto plazo (EMA 12) y largo plazo (EMA 26).
    ema_short = qnta.ema(price, 12)
    ema_long = qnta.ema(price, 26)
    ema_cross = ema_short / ema_long

    # 3. Rate of Change (ROC) - Corto Plazo: Mide el impulso inmediato del precio a muy corto plazo (5 días).
    roc_5d = qnta.roc(price, 5)

    # 4. Aceleración del Impulso (Dual ROC): Mide si el impulso de corto plazo (ROC 10) es mayor que el impulso de largo plazo (ROC 30), indicando aceleración de la tendencia.
    roc_short = qnta.roc(price, 10)
    roc_long = qnta.roc(price, 30)
    dual_roc_accel = roc_short - roc_long

    # 5. Convergencia/Divergencia de Medias Móviles (Diferencia de SMA): Mide el spread entre una media móvil corta (20 días) y una larga (100 días).
    # Reemplazo de BBW. Útil para capturar cambios en la tendencia principal.
    sma_20 = qnta.sma(price, 20)
    sma_100 = qnta.sma(price, 100)
    sma_diff = (sma_20 - sma_100) / price # Normalizado por el precio

    # -----------------------------

    result = xr.concat(
        [trend, macd2_signal.sel(field="close"), volatility, stochastic_d, rsi,
         daily_return1, daily_return2, daily_return3, daily_return4, daily_return5,
         relative_strength_long, ema_cross, roc_5d, dual_roc_accel, sma_diff],
        pd.Index(
            ["trend", "macd", "volatility", "stochastic_d", "rsi",
             "daily_return1", "daily_return2", "daily_return3", "daily_return4", "daily_return5",
             "rs_long", "ema_cross", "roc_5d", "dual_roc", "sma_diff"],
            name = "field"
        )
    )

    return result.transpose("time", "field", "asset")

my_features = get_features(data)
# NOTE: The 'display' function will only work in an interactive environment (like Jupyter/Colab).
try:
    display(my_features.sel(field="trend").to_pandas())
except NameError:
    print("Feature 'trend' (pandas):\n", my_features.sel(field="trend").to_pandas().head())

print(assets[0])
try:
    display(my_features.sel(asset=assets[0]).to_pandas().tail(10))
except NameError:
    print(f"Features for {assets[0]} (tail 10):\n", my_features.sel(asset=assets[0]).to_pandas().tail(10))


def get_target_classes(data):
    """ Target classes for predicting if price goes up or down."""

    price = data.sel(field="close")
    price_future = price.shift(time=-1)

    target_return = (price_future - price)/price

    return target_return


# displaying the target classes:
my_targetclass = get_target_classes(data)
try:
    display(my_targetclass.to_pandas())
except NameError:
    print("Target classes (pandas):\n", my_targetclass.to_pandas().head())



def train_models(data):
    """Create and train the model working on an asset-by-asset basis."""

    asset_name_all = data.coords["asset"].values
    features_all   = get_features(data)
    target_all     = get_target_classes(data)

    models = dict()

    for asset_name in assets:

        # drop missing values:
        target_cur   = target_all.sel(asset=asset_name).dropna(dim="time",how="any")
        features_cur = features_all.sel(asset=asset_name).dropna(dim="time",how="any")

        target_for_learn_df, feature_for_learn_df = xr.align(target_cur, features_cur, join="inner")

        target_for_learn_df  = target_for_learn_df.to_pandas()
        feature_for_learn_df = feature_for_learn_df.to_pandas()

        if len(features_cur.time) < 10:
                continue

        model = get_model()

        try:
            model.fit(feature_for_learn_df.values, target_for_learn_df)
            models[asset_name] = model

        except Exception:
            logging.exception("model training failed")

        try:
            target_pred = model.predict(feature_for_learn_df.values)
            print(asset_name, models[asset_name])
            print('    r2',r2_score(target_for_learn_df, target_pred))
            print('    mean_absolute_error',mean_absolute_error(target_for_learn_df, target_pred))
            print('    explained_variance',explained_variance_score(target_for_learn_df, target_pred))
        except Exception:
            logging.exception("model prediction failed")

    return models



models = train_models(data)

# Define feature names for plotting
feature_names = ["trend", "macd", "volatility", "stochastic_d", "rsi",
                 "daily_return1", "daily_return2", "daily_return3",
                 "daily_return4", "daily_return5"]

## FIX IMPLEMENTED HERE
# Showing which features are more important in predicting:
for k in range(len(assets)):
    asset = assets[k]
    if asset not in models:
        continue

    print(f"\n--- {asset} ---")

    # FIX: Use 'feature_importances_' for RandomForestRegressor
    importance = models[asset].feature_importances_

    # Check that the number of features matches the number of importance scores
    if len(importance) == len(feature_names):
        print("\nFeature Importances (Score):")
        # Combine names and scores and sort for better visualization
        sorted_importance = sorted(zip(feature_names, importance), key=lambda x: x[1], reverse=True)
        for name, score in sorted_importance:
            print(f'  {name:15}: {score:.5f}')

        # Plotting
        plt.figure(figsize=(12, 5))
        plt.bar(feature_names, importance)
        plt.title(f'Feature Importance for {asset}')
        plt.xlabel('Feature')
        plt.ylabel('Importance Score')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    else:
        print(f"Warning: Number of importance scores ({len(importance)}) does not match expected features ({len(feature_names)}) for {asset}.")

# --- End of Fix ---


# loading s&p-500 stock data for testing
data = qndata.stocks.load_spx_data(tail = 365, assets = assets)



def predict_returns(models, data):
    """The model predicts if the price is going up or down.
       The prediction is performed for several days in order to speed up the evaluation."""

    asset_name_all = data.coords["asset"].values
    returns = xr.zeros_like(data.sel(field="close"))

    for asset_name in assets:
        if asset_name in models:
            model = models[asset_name]
            features_all = get_features(data)
            features_cur = features_all.sel(asset=asset_name).dropna(dim="time",how="any")
            if len(features_cur.time) < 1:
                continue
            try:
                returns.loc[dict(asset=asset_name, time=features_cur.time.values)] = model.predict(features_cur.values)
            except KeyboardInterrupt as e:
                raise e
            except Exception:
                logging.exception("model prediction failed")

    return returns



predicted_returns = predict_returns(models, data)


def predict_weights(models, data):
    """The model predicts if the price is going up or down.
       The prediction is performed for several days in order to speed up the evaluation."""

    asset_name_all = data.coords["asset"].values
    returns = xr.zeros_like(data.sel(field="close"))

    for asset_name in assets:
        if asset_name in models:
            model = models[asset_name]
            features_all = get_features(data)
            features_cur = features_all.sel(asset=asset_name).dropna(dim="time",how="any")
            if len(features_cur.time) < 1:
                continue
            try:
                # Assuming 'returns' is a 2D DataArray (time, asset) after this loop
                returns.loc[dict(asset=asset_name, time=features_cur.time.values)] = model.predict(features_cur.values)
            except KeyboardInterrupt as e:
                raise e
            except Exception:
                logging.exception("model prediction failed")


    move = 0.0001
    wdata = []
    for t in returns.time:
        r= returns.sel(time=t).data
        # Ensure r is not empty and has finite values
        if r.size == 0 or not np.all(np.isfinite(r)):
             wdata.append(np.zeros_like(r))
             continue

        sp = r[r > move].sum()
        sn = r[r < -move].sum()
        wt  = []
        for v in r:
            if v > move and sp != 0:
                wt.append(v/sp)
            elif v < -move and sn != 0:
                wt.append(-v/sn)
            else:
                wt.append(0.0)
        wdata.append(np.array(wt))

    # --- FIX APPLIED HERE ---
    # Create the DataArray using only 'time' and 'asset' dimensions/coordinates.
    # The 'returns' array structure already provides the necessary dimensions (time, asset).
    weights = xr.DataArray(
            data = np.array(wdata),
            dims = returns.dims, # ('time', 'asset')
            coords = dict(
                asset = returns.coords['asset'],
                time = returns .coords['time'],
            ),
            attrs = returns.attrs)

    # Note: No need for the .isel(field=0) line anymore as 'field' is not a dimension.
    # qnout.clean will likely expect the weights to be a 2D array of (time, asset).
    # --- END FIX ---

    #weights = qnout.clean(weights, data, "stocks_s&p500")
    return weights



weights = predict_weights(models, data)

for i in range(len(assets)):
    print(assets[i])

    price = data.sel(field="close")
    price_nextday = price.shift(time=-1)
    daily_return = (price_nextday - price)/price
    daily_return =  daily_return.dropna(dim ='time')
    my_targetclass = get_target_classes(data)

    asseti_df= pd.DataFrame()
    asseti_df["price"] = price.to_pandas()[assets[i]]
    asseti_df["price nextday"] = price_nextday.to_pandas()[assets[i]]
    asseti_df["daily_return"] = daily_return.to_pandas()[assets[i]]
    asseti_df["target_return"] = my_targetclass.to_pandas()[assets[i]]
    asseti_df["predicted return"] = predicted_returns.to_pandas()[assets[i]]
    asseti_df["predicted weights"] = weights.to_pandas()[assets[i]]
    try:
        display(asseti_df.tail(10))
    except NameError:
        print(f"Dataframe for {assets[i]} (tail 10):\n", asseti_df.tail(10))


def get_sharpe(data, weights):
    """Calculates the Sharpe ratio"""
    rr = qnstats.calc_relative_return(data, weights)
    sharpe = qnstats.calc_sharpe_ratio_annualized(rr).values[-1]
    return sharpe

sharpe = get_sharpe(data, weights)
print("\nSharpe Ratio:", sharpe)


import qnt.graph as qngraph

statistics = qnstats.calc_stat(data, weights)

try:
    display(statistics.to_pandas().tail())
except NameError:
    print("Statistics (tail):\n", statistics.to_pandas().tail())


performance = statistics.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

try:
    display(statistics[-1:].sel(field = ["sharpe_ratio"]).transpose().to_pandas())
except NameError:
    print("Final Sharpe Ratio:\n", statistics[-1:].sel(field = ["sharpe_ratio"]).transpose().to_pandas())


# check for correlations with existing strategies:
qnstats.print_correlation(weights,data)


# Calculate weights using the backtester:
# NOTE: This block initiates the full backtest process, which can take a while.
weights_backtest = qnbt.backtest_ml(
    train                         = train_models,
    predict                       = predict_weights,
    train_period                  = 5 *365,  # the data length for training in calendar days
    retrain_interval              = 365,  # how often we have to retrain models (calendar days)
    retrain_interval_after_submit = 1,        # how often retrain models after submission during evaluation (calendar days)
    predict_each_day              = False,    # Is it necessary to call prediction for every day during backtesting?
                                              # Set it to True if you suspect that get_features is looking forward.
    competition_type              = "stocks_s&p500",  # competition type
    lookback_period               = 365,                 # how many calendar days are needed by the predict function to generate the output
    start_date                    = "2005-01-01",        # backtest start date
    analyze                       = True,
    build_plots                   = True  # do you need the chart?
)

fetched chunk 1/1 0s
Data loaded 0s


asset,NAS:GOOGL,NAS:MSFT,NAS:NVDA,NAS:TSLA,NYS:JPM,NYS:WMT
time,,,,,,
2020-11-17,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-18,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-19,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-20,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-23,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-11-07,0.569661,-0.106177,0.098675,0.323152,0.105979,-0.017083
2025-11-10,0.472233,-0.105583,0.097905,0.183558,0.110246,0.011449
2025-11-11,0.598519,-0.044593,0.287716,0.290907,0.135244,0.005128


NAS:MSFT


field,trend,macd,volatility,stochastic_d,rsi,daily_return1,daily_return2,daily_return3,daily_return4,daily_return5,rs_long,ema_cross,roc_5d,dual_roc,sma_diff
time,,,,,,,,,,,,,,,
2025-10-31,0.071866,3.570773,0.020372,0.638307,0.541670,-0.015121,-0.029157,-0.000959,0.019849,0.015107,1.023768,1.009777,0.998924,-0.638686,0.024391
2025-11-03,0.021278,3.479575,0.021682,0.468846,0.476800,-0.001506,-0.015121,-0.029157,-0.000959,0.019849,1.007755,1.007767,-1.107695,0.846799,0.023905
2025-11-04,0.016646,3.210583,0.021845,0.290335,0.470842,-0.005222,-0.001506,-0.015121,-0.029157,-0.000959,1.005853,1.005983,-2.726144,-0.455066,0.021966
2025-11-05,0.000045,2.722542,0.021697,0.202728,0.449886,-0.013940,-0.005222,-0.001506,-0.015121,-0.029157,1.000208,1.004104,-5.117420,-1.644791,0.020453
2025-11-06,-0.044650,1.956084,0.021300,0.124369,0.399088,-0.019836,-0.013940,-0.005222,-0.001506,-0.015121,0.986069,1.001484,-6.350291,-1.984306,0.018364
2025-11-07,-0.106177,1.046372,0.021784,0.061982,0.340922,-0.000563,-0.019836,-0.013940,-0.005222,-0.001506,0.966872,0.997856,-5.451156,-2.548221,0.015829
2025-11-10,-0.105583,0.238637,0.020803,0.031241,0.339439,0.018478,-0.000563,-0.019836,-0.013940,-0.005222,0.966809,0.994976,-4.053610,-2.254009,0.014037
2025-11-11,-0.044593,-0.420624,0.020800,0.097387,0.427379,0.005296,0.018478,-0.000563,-0.019836,-0.013940,0.984700,0.994193,-2.133338,-3.130124,0.012478
2025-11-12,-0.026059,-0.912059,0.019969,0.175018,0.450383,0.004836,0.005296,0.018478,-0.000563,-0.019836,0.989778,0.994060,-1.098517,-4.369973,0.011317


asset,NAS:GOOGL,NAS:MSFT,NAS:NVDA,NAS:TSLA,NYS:JPM,NYS:WMT
time,,,,,,
2020-11-17,-0.011932,-0.015761,0.000484,0.101968,-0.007407,-0.001808
2020-11-18,0.010301,0.006348,0.000856,0.025953,0.002690,0.020322
2020-11-19,-0.012618,-0.009557,-0.026227,-0.019348,-0.008567,-0.012424
2020-11-20,-0.005080,-0.001331,0.003992,0.065849,0.028803,0.004593
2020-11-23,0.021035,0.017848,-0.013870,0.064252,0.046237,0.002783
...,...,...,...,...,...,...
2025-11-07,0.040419,0.018478,0.057933,0.036576,0.008529,-0.001657
2025-11-10,0.004171,0.005296,-0.029591,-0.012600,-0.004008,0.009959
2025-11-11,-0.015791,0.004836,0.003313,-0.020518,0.015176,0.000000


...

asset,NAS:AAL,NAS:AAPL,NAS:ABI,NAS:ABKFQ,NAS:ABNB,NAS:ACAS,NAS:ACGL,NAS:ADBE,NAS:ADCT,NAS:ADI
time,,,,,,,,,,
2025-10-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-11-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-11-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-11-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-11-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-11-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-11-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-11-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-11-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Stats:


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2025-10-31,0.224676,-0.004571,0.075808,-0.779526,-0.781036,-0.913655,-0.069262,0.181553,6.0,0.351990,2.351376
2025-11-03,0.224215,-0.002051,0.075802,-0.779978,-0.781036,-0.914773,-0.069342,-0.081497,6.0,0.351945,2.351577
2025-11-04,0.223787,-0.001908,0.075796,-0.780398,-0.781036,-0.915809,-0.069414,0.375538,6.0,0.352020,2.352238
2025-11-05,0.222810,-0.004369,0.075794,-0.781358,-0.781358,-0.918247,-0.069597,0.013751,6.0,0.352046,2.351891
2025-11-06,0.222809,-0.000005,0.075786,-0.781359,-0.781359,-0.918168,-0.069585,1.000000,6.0,0.352045,2.351723
2025-11-07,0.222812,0.000016,0.075779,-0.781355,-0.781359,-0.918077,-0.069571,0.062906,6.0,0.352066,2.352247
2025-11-10,0.224191,0.006187,0.075785,-0.780002,-0.781359,-0.914199,-0.069283,-0.485019,6.0,0.352094,2.352012
2025-11-11,0.223879,-0.001390,0.075778,-0.780308,-0.781359,-0.914933,-0.069332,0.334795,6.0,0.352178,2.351947
2025-11-12,0.222525,-0.006047,0.075782,-0.781637,-0.781637,-0.918300,-0.069590,-0.275216,6.0,0.352295,2.351395


---


100% (5251 of 5251) |####################| Elapsed Time: 0:09:16 Time:  0:09:16
